This notebook reproduces the **neural bud basal surface curvature analysis ** described in the manuscript.  

## Analysis Workflow

1. **ROI Extraction (FIJI)**:  
   - Local basal contours were manually traced in *Fiji* (Segmented Line → Fit Spline).  
   - Traced regions-of-interest (ROIs) were exported as XY coordinates.  

2. **Curvature Estimation**:  
   - Circles were fitted to local contours using least-squares minimization.  
   - The local radius of curvature (*R*) was defined as the mean fitted radius.  
   - Curvature was calculated as the reciprocal (*1/R*).  

3. **Visualization & QC**:  
   - Fitted contours and circle overlays were plotted for manual inspection.  
   - Summary statistics (radius, curvature) were saved to Excel.  

4. **Correlation with Droplet Mobility**:  
   - ROI-derived curvature values were merged with droplet mobility data.  
   - Pearson’s correlation coefficients (*r*, *p*) were computed.  
   - Scatter plots with linear regression overlays visualized the relationship.  

**Outputs**  
- Plots: fitted contours with circle overlays  
- Tables: curvature/radius values per ROI, correlation results  
- Figures: scatter plots showing curvature vs. droplet mobility  


In [ ]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
from glob import glob
from read_roi import read_roi_file  
from read_roi import read_roi_zip
from scipy.stats import pearsonr
from scipy.optimize import leastsq

### Step 1: Fit local surface contours with circles

In [ ]:
base_folder = "input-folder-name/" 

results = []

# --------- Step 1: Convert all .roi files to .txt ----------
roi_files = glob(os.path.join(base_folder, "*.roi"))

for roi_path in roi_files:
    try:
        roi_data = read_roi_file(roi_path)
        roi_name = list(roi_data.keys())[0]
        roi = roi_data[roi_name]

        x = np.array(roi['x'])
        y = np.array(roi['y'])

        # Save to txt file
        coords = np.column_stack((x, y))
        txt_path = os.path.splitext(roi_path)[0] + ".txt"
        np.savetxt(txt_path, coords, fmt="%.3f", delimiter="\t")

    except Exception as e:
        print(f"❌ Failed to convert {roi_path}: {e}")

# --------- Step 2: Batch process all txt files ----------
txt_files = glob(os.path.join(base_folder, "*.txt"))

for txt_path in txt_files:
    fname = os.path.basename(txt_path)
    
    try:
        coords = np.loadtxt(txt_path)
        x = coords[:, 0]
        y = coords[:, 1]

        # Circle fitting
        def calc_R(xc, yc):
            return np.sqrt((x - xc)**2 + (y - yc)**2)
        
        def f_2(c):
            Ri = calc_R(*c)
            return Ri - Ri.mean()

        x_m, y_m = np.mean(x), np.mean(y)
        center_estimate = x_m, y_m

        center, _ = leastsq(f_2, center_estimate)
        xc, yc = center
        Ri = calc_R(xc, yc)
        R = Ri.mean()
        curvature = 1 / R

        results.append({
            'index': fname,
            'radius_pixels': R,
            'curvature_1_per_pixel': curvature
        })

        # Plot
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.plot(x, y, 'bo', label='ROI points')
        circle = plt.Circle((xc, yc), R, color='r', fill=False, linestyle='--', label='Fitted circle')
        ax.add_patch(circle)
        ax.plot(xc, yc, 'kx', label='Center')
        ax.set_aspect('equal')
        ax.set_title(f"Circle Fit: {fname}")
        ax.set_xlabel("X (pixels)")
        ax.set_ylabel("Y (pixels)")
        ax.legend()
        ax.grid(True)

        plot_path = os.path.join(base_folder, f"{os.path.splitext(fname)[0]}_fit.png")
        plt.savefig(plot_path, dpi=300)
        plt.close()

    except Exception as e:
        print(f"❌ Error processing {fname}: {e}")

# --------- Step 3: Save summary to Excel ----------
df = pd.DataFrame(results)
excel_path = os.path.join(base_folder, "curvature_summary_pixels.xlsx")
df.to_excel(excel_path, index=False)

print(f"✅ Done! Processed {len(results)} files.")
print(f"📁 Excel saved to: {excel_path}")

### Step2: check correlation

In [ ]:
file_path = f'{base_folder}curvature_summary_pixels.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1')

output_folder_path = 'output_folder_name' 

In [ ]:
required_cols = ['mobility', 'curvature_1_per_micron']
if not all(col in df.columns for col in required_cols):
    raise ValueError(f"Missing one or more required columns: {required_cols}")

df_clean = df.dropna(subset=required_cols)

# === Compute Pearson correlation
r, p_value = pearsonr(df_clean['mobility'], df_clean['curvature_1_per_micron'])
print(f"📈 Pearson correlation: r = {r:.3f}, p = {p_value:.3g}")

# Step 5: Plot
plt.figure(figsize=(4, 3))
plt.rcParams['font.family'] = 'Arial'
sns.regplot(
    x='curvature_1_per_micron',
    y='mobility',
    data=df_clean,
    scatter_kws={'s': 20, 'alpha': 0.6, 'color': 'black'},
    line_kws={'color': 'red', 'linewidth': 2},
    ci=None 
)

plt.title(f"Mobility vs. Curvature\nPearson r = {r:.2f}, p = {p_value:.2g}")
plt.xlabel("Curvature (1/μm)")
plt.ylabel("Mobility")

plt.tight_layout()
plt.savefig(f'{folder_path}correlation5.pdf', format='pdf')
plt.show()